# Object Oriented Programming

Objects are _very_ powerful, and you can harness this power for your own designs!

## Writing a class

Writing a class should be second nature, and very natural to you - just as natural as writing a function!

In [ ]:
def f(x: float) -> float:
    return x**2

print(f"{f(3) = }")

In [ ]:
class F:
    def __call__(self, x: float) -> float:
        return x**2
    
f = F()
print(f"{f(3) = }")



### Data + functions

They allow you to bundle data with the functions that run on them. In a language like Python without (much) type overloading, this is important for design (and good for tab completion).

In [ ]:
# Built in for Python 3.7+, install otherwise
from dataclasses import dataclass

@dataclass
class Vector:
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y
        
    def mag(self) -> float:
        return (self.x**2 + self.y**2)**.5

> Note: developing a class that represents data has a bit of boilerplate involved, so attrs (third-part) or dataclasses (first party in Python 3.7, thirdparty for 3.6) has a trick to make them _much more easily_, and automatically add things like nice repr's. We'll cover the parts of the syntax here later.

In [ ]:
# Built in for Python 3.7+, install otherwise
from dataclasses import dataclass

@dataclass
class Vector:
    x: float
    y: float
        
    def mag(self) -> float:
        return (self.x**2 + self.y**2)**.5

In [ ]:
v = Vector(3,4)
v.mag()

### Functors

I just told you it was a bad idea to set something outside your local scope, and often not even a good idea to just view something outside the local scope. So how do you write something that has scope? Use a class as a functor! Compare this:

In [ ]:
_start = 0
def incr() -> int:
    global _start
    _start += 1
    return _start

In [ ]:
incr()

In [ ]:
incr()

With this:

In [ ]:
class Incr:
    def __init__(self, incr: int = 0) -> None:
        self.incr = incr
    def __call__(self) -> int:
        self.incr += 1
        return self.incr

In [ ]:
incr = Incr()
incr()

In [ ]:
incr()

This is explicit, clear, I can have multiple instances without having them interfere, I can see exactly what's going on without having to trace down a global, and I can even set the default value when I make the Incr instance!

### Modularity

Let's say you have an algorithm with three parts. If you make a class that calls three member functions, you can allow a user to replace just one of the functions, and use the original first two. Classes are great for _originization_ and _code reuse_ because of this.

In [ ]:
class RunSomethingHard:
    def part1(self) -> None:
        print("Working hard")
    def part2(self) -> None:
        print("Working harder")
    def part3(self) -> None:
        print("That was hard!")
    def run(self) -> None:
        self.part1()
        self.part2()
        self.part3()

In [ ]:
inst = RunSomethingHard()
inst.run()

Now, look at how I can swap out part of the calculation without rewriting from scratch!

In [ ]:
class NewRunSomethingHard(RunSomethingHard):
    def part2(self) -> None:
        print("Nah, this is easy")

In [ ]:
inst = NewRunSomethingHard()
inst.run()

### DSL (Domain Specific Language)

You can customize almost every behavoir of a class to make them very natural for whatever you are doing.

In [ ]:
class Path(str):
    def __truediv__(self, other):
        return self.__class__(f"{self}/{other}")

In [ ]:
Path(".") / "myfile" / "program.py"

> Just in case you want to make a Path class like the one above - don't, use pathlib instead. We could ahve written `self.__class__` as `Path`, but then this would not subclass correctly and besides, using the class name inside the class is ugly and makes it harder to rename. If you return a normal string, then you can't keep applying `/`.
>
> Also, I left off type annoations for this example, as to do them properly I need to use a TypeVar.

### Mixins (advanced)

If you follow good practices, you can even make collections of behaviors and mix them into other classes - specifically, a mixin should not have an `__init__` or any new datamembers. (The second requirement is more important than the first, if you are carful to use `super()`. Let's rewrite the last example with mixins:

In [ ]:
class PathMixin:
    def __truediv__(self, other):
        return self.__class__(f"{self}/{other}")
    
class Path(str, PathMixin):
    pass

Path(".") / "myfile" / "program.py"

> Potentially show demo here?

### Other: ABC, Protocols, and more

Other useful things to look into are ABCs (Abstract Base Classes) and Protocols. These let you a) require certain methods be implemented by users, and b) formalize "duck typing". ABC's are a run-time feature, and kind of half-broken; you have to instanciate an ABC class to get the benefit of the checking. Protocols are a "type-check time" feature, and are far better and don't require special inheratince, but only are enforced by type checkers (see a later section!).

## Design considerations

Object oriented programming has been known to make it easy to create spegetti messes of code. The following tips will help you not fall into the trap and end up with poorly designed code. "Make it a class", by itself, will not magically make your code better.

### Modular design

You should break your code into _concepts_, and classes should help map those concepts to the computer. Different components of a detector might be classes, with an instance for each component. A vector, a URL, a remote data source, etc. You might have a class representing a unit of an analysis, and use either inheritance (okay) or a protocol (better, reduces coupling) to have real data processing vs. simulation generation. Etc.

### Unit test

We will mention testing later, and there is another course on it, but I'm focusing on the word _unit_. You need to be able to run your classes standalone, in unit tests, and not only inplace. This keeps the design modular - you will resist the desire to make a class that needs a class to make another class inside a class that only works with the file that sits on your work laptop, etc. And you'll be free to redesign parts without having to worry about everything breaking down.

Always use PyTest for unit testing.